In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold#, KFold
# from sklearn.metrics import roc_auc_score
# from fastai.vision.all import *
import wandb
# import timm

In [2]:
# approach = ''

In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchinfo import summary
# # from fastai.callback.wandb import *
# from torch.cuda.amp import autocast, GradScaler

In [4]:
# from fastai.vision.all import *
from fastai.tabular.all import *
# from fastai.callback.wandb import WandbCallback

In [5]:
# import xgboost as xgb
from xgboost import XGBRegressor, XGBRFRegressor
# from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier

In [6]:
%config Completer.use_jedi = False

In [7]:
set_seed(42, reproducible=True)

In [8]:
datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/202108_august/')

In [9]:
df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)

In [10]:
# df.isnull().sum().any() # False

In [11]:
# df.info(verbose=True, null_counts=True)

In [12]:
# df.describe()

In [13]:
y = df.loss

In [14]:
features = [x for x in df.columns if x != 'loss']

In [15]:
X = df[features]

In [16]:
len(X)

250000

In [17]:
df.head()

          f0   f1        f2        f3          f4        f5        f6  \
id                                                                      
0  -0.002350   59  0.766739 -1.350460     42.2727  16.68570   30.3599   
1   0.784462  145 -0.463845 -0.530421  27324.9000   3.47545  160.4980   
2   0.317816   19 -0.432571 -0.382644   1383.2600  19.71290   31.1026   
3   0.210753   17 -0.616454  0.946362   -119.2530   4.08235  185.2570   
4   0.439671   20  0.968126 -0.092546     74.3020  12.30650   72.1860   

          f7         f8       f9  ...        f91        f92      f93  \
id                                ...                                  
0   1.267300   0.392007  1.09101  ...  -42.43990  26.854000  1.45751   
1   0.828007   3.735860  1.28138  ... -184.13200   7.901370  1.70644   
2  -0.515354  34.430800  1.24210  ...    7.43721  37.218100  3.25339   
3   1.383310 -47.521400  1.09130  ...    9.66778   0.626942  1.49425   
4  -0.233964  24.399100  1.10151  ...  290.65700  15.604

In [18]:
len(y), len(y.value_counts())

(250000, 43)

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# from sklearn.ensemble import RandomForestRegressor

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold#, KFold
# from sklearn.metrics import roc_auc_score
# from fastai.vision.all import *
import wandb
from wandb.xgboost import wandb_callback
# import timm

In [22]:
# model = RandomForestRegressor(random_state=42, min_samples_leaf=2, n_jobs=-1, n_estimators=100)
model = XGBRegressor(n_estimators=200, learning_rate=0.1, n_jobs=-1, verbosity=1, random_state=42)

In [23]:
model.get_xgb_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'monotone_constraints': None,
 'n_jobs': -1,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': 1}

In [24]:
model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 200,
 'n_jobs': -1,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': 1}

In [25]:
len(model.get_xgb_params()) == len(model.get_params())

False

In [26]:
len(model.get_xgb_params())

24

In [27]:
len(model.get_params())

27

In [28]:
[param for param in model.get_params().keys() if param not in model.get_xgb_params().keys()]

['importance_type', 'missing', 'n_estimators']

In [29]:
# import wandb
wandb.init(
    project="202108_Kaggle_tabular_playground",
    save_code=True,
    tags=['XGBoost'],
    name='202108062041_RandomForestRegressor',
    notes='First run of XGBRegressor',
    config={
        'library': 'xgboost',
        'model': model,
        'params': model.get_params()
    }
        
    #                'initial_learning_rate':lr_steep,
#        'metric':'roc_auc',
#        'n_epochs': n_epochs,
#        'batch_size': batch_size,
#        'optimizer':optimizer,
#        'loss_func':loss_func,
#        'mixup_applied': mixup_applied,
#        'mixup_alpha': mixup_alpha,
#        'architecture':arch,
#        'model_source': model_source,
#        'pretrained': pretrained,
#        'transform_types': transform_types,
#        'transform_libraries': transform_libraries,
#        'cv_transforms': cv_transforms_applied,
#        'dsp_transforms': dsp_transforms_applied,
#        'trainset_type':trainset_type,
#        'k-fold': fold,
#        'k-fold_total': k_folds,
#        'k-fold_strategy': 'stratified',
#        'lr_strategy': 'valley',
#        'lr': lr
)

In [30]:
?model.fit

In [31]:
model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
# model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=1)

In [32]:
# wandb.sklearn.plot_regressor(model, X_train, X_valid, y_train, y_valid,  model_name='RandomForestRegressor', )

In [33]:
# y_pred = model.predict(X_valid)

In [34]:
# from sklearn.model_selection import cross_val_score

In [35]:
# scores = cross_val_score(model, X, y, cv=5, n_jobs=-1)

In [36]:
# model.get_depth()

In [37]:
from sklearn.metrics import mean_squared_error

In [38]:
y_valid_preds = model.predict(X_valid)

In [39]:
mse = mean_squared_error(y_valid, y_valid_preds)

In [40]:
mse

61.500556614829804

In [41]:
rmse = math.sqrt(mse)
rmse

7.842229059064126

In [42]:
# rmses = [math.sqrt(abs(score)) for score in scores]
# rmses

In [43]:
test_df = pd.read_csv(datapath/'test.csv', index_col='id', low_memory=False)

In [44]:
test_df.head()

              f0   f1        f2        f3          f4        f5         f6  \
id                                                                           
250000  0.812665   15 -1.239120 -0.893251    295.5770  15.87120   23.04360   
250001  0.190344  131 -0.501361  0.801921     64.8866   3.09703  344.80500   
250002  0.919671   19 -0.057382  0.901419  11961.2000  16.39650  273.24000   
250003  0.860985   19 -0.549509  0.471799   7501.6000   2.80698   71.08170   
250004  0.313229   89  0.588509  0.167705   2931.2600   4.34986    1.57187   

              f7         f8       f9  ...       f90        f91       f92  \
id                                    ...                                  
250000  0.942256  29.898000  1.11394  ...  0.446389   -422.332  -1.44630   
250001  0.807194  38.421900  1.09695  ...  0.377179  10352.200  21.06270   
250002 -0.003300  37.940000  1.15222  ...  0.990140   3224.020  -2.25287   
250003  0.792136   0.395235  1.20157  ...  1.396880   9689.760  14.77150 

In [45]:
X_test = test_df[features] # this is just for naming consistency

In [46]:
y_test_preds = model.predict(X_test)

In [47]:
sample_df = pd.read_csv(datapath/'sample_submission.csv')

In [48]:
sample_df.loc[:, 'loss'] = y_test_preds

In [49]:
sample_df.head()

       id      loss
0  250000  7.328520
1  250001  4.610067
2  250002  7.277142
3  250003  7.030011
4  250004  7.187167

In [50]:
sample_df.to_csv('202108062038_XGBoost.csv', index=False)

In [51]:
wandb.finish()